## Pacakge & Data Import

In [11]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm.notebook import tqdm
import time
from torchsummary import summary
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
import pandas as pd
import seaborn as sns
import os

In [12]:
# import labels for gudhi shapes
# gudhi_shape_labels = np.genfromtxt('Gudhi Shape Dataset/shape_labels.csv', delimiter=',', skip_header=1)
# gudhi_shape_labels = gudhi_shape_labels.astype(int)[:,2]
# print(len(gudhi_shape_labels))

In [13]:
# FOR PARTIAL DATASET MANUALLY CREATING LABELS

gudhi_shapes = np.array([])

base = 'Gudhi Shape Dataset Non-uniform and Uniform Sampling'

# Check if the required files exist for each shape
for i in range(1000, 4261):
    points_file = f'{base}/shape_{i}_points.csv'
    laplacian_file = f'{base}/shape_{i}_laplacian.csv'
    vr_persistence_image_file = f'{base}/shape_{i}_vr_persistence_image.csv'
    abstract_persistence_image_file = f'{base}/shape_{i}_abstract_persistence_image.csv'

    if (os.path.exists(points_file) and
        os.path.exists(laplacian_file) and
        os.path.exists(vr_persistence_image_file) and
        os.path.exists(abstract_persistence_image_file)):
        gudhi_shapes = np.append(gudhi_shapes, i)
    

gudhi_shapes = gudhi_shapes.astype(int)

gudhi_shape_labels = np.array([])

# if shape[i] % 1000 < 500, gudhi_shape_labels[i] = 1, else 0

for i in range(len(gudhi_shapes)):
    if gudhi_shapes[i] % 1000 < 500:
        gudhi_shape_labels = np.append(gudhi_shape_labels, 1)
    else:
        gudhi_shape_labels = np.append(gudhi_shape_labels, 0)

In [14]:
len(gudhi_shapes)

2219

In [15]:
len(gudhi_shape_labels)

2219

In [16]:
seed_value = 221
np.random.seed(seed_value)

num_samples = len(gudhi_shape_labels) # currently set to full dataset

# Generate random indices
random_indices = np.random.choice(len(gudhi_shape_labels), size=num_samples, replace=False)
base = 'Gudhi Shape Dataset Non-uniform and Uniform Sampling'
# Select the corresponding data and labels
gudhi_laplacians = []
gudhi_vr_persistence_images = []
gudhi_abstract_persistence_images = []
gudhi_selected_labels = []

for i in random_indices:
    gudhi_laplacians.append(np.genfromtxt(f'{base}/shape_{gudhi_shapes[i]}_laplacian.csv', delimiter=',', skip_header=0))
    gudhi_vr_persistence_images.append(np.genfromtxt(f'{base}/shape_{gudhi_shapes[i]}_vr_persistence_image.csv', delimiter=',', skip_header=0))
    gudhi_abstract_persistence_images.append(np.genfromtxt(f'{base}/shape_{gudhi_shapes[i]}_abstract_persistence_image.csv', delimiter=',', skip_header=0))
    gudhi_selected_labels.append(gudhi_shape_labels[i])

# Convert selected labels to NumPy array
gudhi_selected_labels = np.array(gudhi_selected_labels)

# Print summary
print(f"Randomly selected {num_samples} samples.")
print(f"Shape of laplacians: {np.array(gudhi_laplacians).shape}")
print(f"Shape of VR persistence images: {np.array(gudhi_vr_persistence_images).shape}")
print(f"Shape of abstract persistence images: {np.array(gudhi_abstract_persistence_images).shape}")
print(f"Shape of selected labels: {gudhi_selected_labels.shape}")

Randomly selected 2219 samples.
Shape of laplacians: (2219, 1000, 1000)
Shape of VR persistence images: (2219, 100, 100)
Shape of abstract persistence images: (2219, 100, 100)
Shape of selected labels: (2219,)


In [17]:
# import labels for medical shapes
medical_shape_labels = np.genfromtxt('Medical Dataset/shape_labels.csv', delimiter=',', skip_header=1)
medical_shape_labels = medical_shape_labels.astype(int)[:,2]
print(len(medical_shape_labels))

162


In [18]:
seed_value = 221
np.random.seed(seed_value)

num_samples = 162 # currently set to full dataset

# Generate random indices
random_indices = np.random.choice(len(medical_shape_labels), size=num_samples, replace=False)
base = 'Medical Dataset/'
# Select the corresponding data and labels
medical_laplacians = []
medical_vr_persistence_images = []
medical_abstract_persistence_images = []
medical_selected_labels = []

for i in random_indices:
    medical_laplacians.append(np.genfromtxt(f'{base}/shape_{i}_laplacian.csv', delimiter=',', skip_header=0))
    medical_vr_persistence_images.append(np.genfromtxt(f'{base}/shape_{i}_vr_persistence_image.csv', delimiter=',', skip_header=0))
    medical_abstract_persistence_images.append(np.genfromtxt(f'{base}/shape_{i}_abstract_persistence_image.csv', delimiter=',', skip_header=0))
    medical_selected_labels.append(medical_shape_labels[i])

# Convert selected labels to NumPy array
medical_selected_labels = np.array(medical_selected_labels)

# Print summary
print(f"Randomly selected {num_samples} samples.")
print(f"Shape of laplacians: {np.array(medical_laplacians).shape}")
print(f"Shape of VR persistence images: {np.array(medical_vr_persistence_images).shape}")
print(f"Shape of abstract persistence images: {np.array(medical_abstract_persistence_images).shape}")
print(f"Shape of selected labels: {medical_selected_labels.shape}")

Randomly selected 162 samples.
Shape of laplacians: (162, 1000, 1000)
Shape of VR persistence images: (162, 100, 100)
Shape of abstract persistence images: (162, 100, 100)
Shape of selected labels: (162,)


## Define data class

In [19]:
class ShapeDataset(torch.utils.data.Dataset):
    def __init__(self, data, labels):
        self.data = [torch.tensor(d, dtype=torch.float32).unsqueeze(0) for d in data]
        self.labels = torch.tensor(labels, dtype=torch.long)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]


## Data Splitting

In [20]:
train_ratio = 0.8
valid_ratio = 0.1
test_ratio = 0.1

# Split Gudhi Laplacians
train_data_gudhi_laplacians, test_data_gudhi_laplacians, train_labels_gudhi, test_labels_gudhi = train_test_split(
    gudhi_laplacians, gudhi_selected_labels, test_size=(1 - train_ratio), random_state=42
)
valid_data_gudhi_laplacians, test_data_gudhi_laplacians, valid_labels_gudhi, test_labels_gudhi = train_test_split(
    test_data_gudhi_laplacians, test_labels_gudhi, test_size=(test_ratio / (valid_ratio + test_ratio)), random_state=42
)

# Split Medical Laplacians (valid/test split only, since all are used for validation/testing)
valid_data_medical_laplacians, test_data_medical_laplacians, valid_labels_medical, test_labels_medical = train_test_split(
    medical_laplacians, medical_selected_labels, test_size=0.5, random_state=42
)

# Combine validation sets for Laplacians and labels
valid_laplacians = valid_data_gudhi_laplacians
valid_labels = valid_labels_gudhi

# Combine test sets for completeness
test_laplacians = test_data_gudhi_laplacians
test_labels = test_labels_gudhi

# Print a summary
print(f"Laplacians Train data size: {len(train_data_gudhi_laplacians)}")
print(f"Laplacians Validation data size: {len(valid_laplacians)}")
print(f"Laplacians Test data size: {len(test_laplacians)}")


# Split VR Persistence Images
train_data_gudhi_vr_persistence_images, test_data_gudhi_vr_persistence_images, train_labels_gudhi_check, test_labels_gudhi_check = train_test_split(
    gudhi_vr_persistence_images, gudhi_selected_labels, test_size=(1 - train_ratio), random_state=42
)
valid_data_gudhi_vr_persistence_images, test_data_gudhi_vr_persistence_images, valid_labels_gudhi_check, test_labels_gudhi_check = train_test_split(
    test_data_gudhi_vr_persistence_images, test_labels_gudhi_check, test_size=(test_ratio / (valid_ratio + test_ratio)), random_state=42
)

# Split Medical VR Persistence Images (valid/test split only, since all are used for validation/testing)
valid_data_medical_vr_persistence_images, test_data_medical_vr_persistence_images, valid_labels_medical_check, test_labels_medical_check = train_test_split(
    medical_vr_persistence_images, medical_selected_labels, test_size=0.5, random_state=42
)

# Combine validation sets for VR Persistence Images and labels
valid_vr_persistence_images = valid_data_gudhi_vr_persistence_images
valid_vr_labels = valid_labels_gudhi

# Combine test sets for VR Persistence Images and labels
test_vr_persistence_images = test_data_gudhi_vr_persistence_images
test_vr_labels = test_labels_gudhi


# Split Abstract Persistence Images
train_data_gudhi_abstract_persistence_images, test_data_gudhi_abstract_persistence_images, train_labels_gudhi_check, test_labels_gudhi_check = train_test_split(
    gudhi_abstract_persistence_images, gudhi_selected_labels, test_size=(1 - train_ratio), random_state=42
)
valid_data_gudhi_abstract_persistence_images, test_data_gudhi_abstract_persistence_images, valid_labels_gudhi_check, test_labels_gudhi_check = train_test_split(
    test_data_gudhi_abstract_persistence_images, test_labels_gudhi_check, test_size=(test_ratio / (valid_ratio + test_ratio)), random_state=42
)

# Split Medical Abstract Persistence Images (valid/test split only, since all are used for validation/testing)
valid_data_medical_abstract_persistence_images, test_data_medical_abstract_persistence_images, valid_labels_medical_check, test_labels_medical_check = train_test_split(
    medical_abstract_persistence_images, medical_selected_labels, test_size=0.5, random_state=42
)

# Combine validation sets for Abstract Persistence Images and labels
valid_abstract_persistence_images = valid_data_gudhi_abstract_persistence_images
valid_abstract_labels = valid_labels_gudhi

# Combine test sets for Abstract Persistence Images and labels
test_abstract_persistence_images = test_data_gudhi_abstract_persistence_images
test_abstract_labels = test_labels_gudhi


Laplacians Train data size: 1775
Laplacians Validation data size: 222
Laplacians Test data size: 222


## CNN Definitions

In [21]:
class CNN(nn.Module):
    def __init__(self, input_shape, num_classes=2):
        super(CNN, self).__init__()
        # Convolutional Layers
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        
        # Pooling Layer
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # Adaptive Pooling to resize to 100x100
        self.adaptive_pool = nn.AdaptiveAvgPool2d((100, 100))
        
        # Dynamically calculate input size to fc1
        self.feature_size = self._get_feature_size(input_shape)
        
        # Fully Connected Layers
        self.fc1 = nn.Linear(self.feature_size, 128)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(128, num_classes)

    def _get_feature_size(self, input_shape):
        # Create a dummy input to calculate size after conv and pooling
        dummy_input = torch.zeros(1, 1, *input_shape)
        x = self.pool(F.relu(self.conv1(dummy_input)))
        x = self.pool(F.relu(self.conv2(x)))
        
        # Apply adaptive pooling to get 100x100 size
        x = self.adaptive_pool(x)
        return x.numel()  # Number of elements after flattening

    def forward(self, x):
        # Apply convolutional layers with pooling
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        
        # Apply adaptive pooling to resize to 100x100
        x = self.adaptive_pool(x)
        
        # Flatten and pass through fully connected layers
        x = torch.flatten(x, start_dim=1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)


In [22]:
class DualInputCNN(nn.Module):
    def __init__(self, input_shape1, input_shape2, num_classes=2):
        super(DualInputCNN, self).__init__()

        # Laplacian input path with additional pooling to reduce to 100x100
        self.conv1_lap = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
        self.conv2_lap = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.pool_lap = nn.MaxPool2d(2, 2)  # Reduce spatial dimensions
        self.adaptive_pool_lap = nn.AdaptiveAvgPool2d((100, 100))  # Resize to 100x100
        
        # Persistence image input path (no pooling)
        self.conv1_pers = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
        self.conv2_pers = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.adaptive_pool_pers = nn.AdaptiveAvgPool2d((100, 100))  # Resize to 100x100
        
        # Fully connected layers
        self.fc1 = nn.Linear(32 * 100 * 100 + 32 * 100 * 100, 128)  # Adjusted for 100x100 input
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x1, x2):
        # Laplacians path (downsampling to 100x100)
        x1 = F.relu(self.conv1_lap(x1))
        x1 = self.pool_lap(x1)  # First pool: 250x250 -> 125x125
        x1 = F.relu(self.conv2_lap(x1))
        x1 = self.pool_lap(x1)  # Second pool: 125x125 -> 62x62
        x1 = self.adaptive_pool_lap(x1)  # Resize to 100x100
        
        # Persistence images path (no pooling)
        x2 = F.relu(self.conv1_pers(x2))
        x2 = F.relu(self.conv2_pers(x2))
        x2 = self.adaptive_pool_pers(x2)  # Ensure persistence images are 100x100
        
        # Concatenate along dim=1 (channels)
        x = torch.cat((x1, x2), dim=1)  # Concatenates the outputs along the channel axis

        # Flatten for fully connected layer
        x = torch.flatten(x, start_dim=1)
        
        # Fully connected layers
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        
        return F.log_softmax(x, dim=1)


In [23]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Training on {device}")

Training on cpu


## Data Preparation

In [24]:
# Convert data to PyTorch Datasets
train_dataset_laplacians = ShapeDataset(train_data_gudhi_laplacians, train_labels_gudhi)
valid_dataset_laplacians = ShapeDataset(valid_laplacians, valid_labels)
test_dataset_laplacians = ShapeDataset(test_laplacians, test_labels)

train_dataset_vr = ShapeDataset(train_data_gudhi_vr_persistence_images, train_labels_gudhi)
valid_dataset_vr = ShapeDataset(valid_vr_persistence_images, valid_vr_labels)
test_dataset_vr = ShapeDataset(test_vr_persistence_images, test_vr_labels)

train_dataset_abstract = ShapeDataset(train_data_gudhi_abstract_persistence_images, train_labels_gudhi)
valid_dataset_abstract = ShapeDataset(valid_abstract_persistence_images, valid_abstract_labels)
test_dataset_abstract = ShapeDataset(test_abstract_persistence_images, test_abstract_labels)

# Define DataLoaders
batch_size = 32

train_loader_laplacians = torch.utils.data.DataLoader(train_dataset_laplacians, batch_size=batch_size, shuffle=False)
valid_loader_laplacians = torch.utils.data.DataLoader(valid_dataset_laplacians, batch_size=batch_size, shuffle=False)
test_loader_laplacians = torch.utils.data.DataLoader(test_dataset_laplacians, batch_size=batch_size, shuffle=False)

train_loader_vr = torch.utils.data.DataLoader(train_dataset_vr, batch_size=batch_size, shuffle=False)
valid_loader_vr = torch.utils.data.DataLoader(valid_dataset_vr, batch_size=batch_size, shuffle=False)
test_loader_vr = torch.utils.data.DataLoader(test_dataset_vr, batch_size=batch_size, shuffle=False)

train_loader_abstract = torch.utils.data.DataLoader(train_dataset_abstract, batch_size=batch_size, shuffle=False)
valid_loader_abstract = torch.utils.data.DataLoader(valid_dataset_abstract, batch_size=batch_size, shuffle=False)
test_loader_abstract = torch.utils.data.DataLoader(test_dataset_abstract, batch_size=batch_size, shuffle=False)

## Model Instantiation

In [25]:
# For single-input CNN (Laplacians)
input_shape = train_data_gudhi_laplacians[0].shape
num_classes = 2 # binary classification

model_single_laplacians = CNN(input_shape, num_classes)

# For dual-input CNNs (Laplacians + VR Persistence Images, Laplacians + Abstract Persistence Images)
input_shape1 = train_data_gudhi_laplacians[0].shape
input_shape2 = train_data_gudhi_vr_persistence_images[0].shape
input_shape3 = train_data_gudhi_abstract_persistence_images[0].shape

model_dual_lap_vr = DualInputCNN(input_shape1, input_shape2, num_classes)
model_dual_lap_abstract = DualInputCNN(input_shape1, input_shape3, num_classes)

## Training, Validation, and Testing Functions

In [26]:
def train_single_input(model, dataloader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    correct = 0
    progress_bar = tqdm(dataloader, desc="Training", leave=False)
    
    for inputs, labels in progress_bar:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        preds = torch.argmax(outputs, dim=1)
        correct += (preds == labels).sum().item()
        
        progress_bar.set_postfix(loss=loss.item())
    
    accuracy = correct / len(dataloader.dataset)
    return total_loss / len(dataloader.dataset), accuracy


def validate_single_input(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0
    correct = 0
    progress_bar = tqdm(dataloader, desc="Validating", leave=False)
    
    with torch.no_grad():
        for inputs, labels in progress_bar:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            preds = torch.argmax(outputs, dim=1)
            correct += (preds == labels).sum().item()
            
            progress_bar.set_postfix(loss=loss.item())
    
    accuracy = correct / len(dataloader.dataset)
    return total_loss / len(dataloader.dataset), accuracy


def train_dual_input(model, dataloader1, dataloader2, optimizer, criterion, device):
    model.train()
    total_loss = 0
    correct = 0
    progress_bar = tqdm(zip(dataloader1, dataloader2), desc="Training (Dual Input)", leave=False, total=min(len(dataloader1), len(dataloader2)))
    
    for (inputs1, labels1), (inputs2, labels2) in progress_bar:
        inputs1, labels1 = inputs1.to(device), labels1.to(device)
        inputs2, labels2 = inputs2.to(device), labels2.to(device)
        
        if not torch.equal(labels1, labels2):
            print("Labels mismatch in dual-input training! Skipping batch.")
            continue
        
        optimizer.zero_grad()
        outputs = model(inputs1, inputs2)
        loss = criterion(outputs, labels1)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        preds = torch.argmax(outputs, dim=1)
        correct += (preds == labels1).sum().item()
        
        progress_bar.set_postfix(loss=loss.item())
    
    accuracy = correct / len(dataloader1.dataset)
    return total_loss / len(dataloader1.dataset), accuracy


def validate_dual_input(model, valid_loader_laplacians, valid_loader_vr, criterion, device='cuda'):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for (inputs1, _), (inputs2, targets) in zip(valid_loader_laplacians, valid_loader_vr):
            inputs1, inputs2, targets = inputs1.to(device), inputs2.to(device), targets.to(device)
            
            outputs = model(inputs1, inputs2)
            loss = criterion(outputs, targets)
            running_loss += loss.item()
            
            if outputs.shape[-1] > 1:
                _, predicted = torch.max(outputs, 1)
            else:  # Binary classification
                predicted = (outputs > 0.5).float()
            
            total += targets.size(0)
            correct += (predicted == targets).sum().item()

    avg_loss = running_loss / len(valid_loader_laplacians)
    accuracy = correct / total

    return avg_loss, accuracy

def test_single_input(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    all_preds = []
    all_labels = []
    progress_bar = tqdm(dataloader, desc="Testing", leave=False)

    with torch.no_grad():
        for inputs, labels in progress_bar:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            preds = torch.argmax(outputs, dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

            progress_bar.set_postfix(loss=loss.item())

    avg_loss = total_loss / len(dataloader.dataset)
    accuracy = correct / total
    precision = precision_score(all_labels, all_preds, average='binary')
    recall = recall_score(all_labels, all_preds, average='binary')
    f1 = f1_score(all_labels, all_preds, average='binary')

    metrics_df = pd.DataFrame({
        'Loss': [avg_loss],
        'Accuracy': [accuracy],
        'Precision': [precision],
        'Recall': [recall],
        'F1 Score': [f1]
    })

    cm = confusion_matrix(all_labels, all_preds)

    print(f"Test Loss: {avg_loss:.4f}, Test Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")
    return metrics_df, cm


def test_dual_input(model, dataloader1, dataloader2, criterion, device='cuda'):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    all_preds = []
    all_labels = []
    progress_bar = tqdm(zip(dataloader1, dataloader2), desc="Testing (Dual Input)", leave=False, total=min(len(dataloader1), len(dataloader2)))

    with torch.no_grad():
        for (inputs1, _), (inputs2, targets) in progress_bar:
            inputs1, inputs2, targets = inputs1.to(device), inputs2.to(device), targets.to(device)

            outputs = model(inputs1, inputs2)
            loss = criterion(outputs, targets)
            running_loss += loss.item()

            if outputs.shape[-1] > 1:
                _, predicted = torch.max(outputs, 1)
            else:  # Binary classification
                predicted = (outputs > 0.5).float()

            total += targets.size(0)
            correct += (predicted == targets).sum().item()

            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(targets.cpu().numpy())

    avg_loss = running_loss / len(dataloader1.dataset)
    accuracy = correct / total
    precision = precision_score(all_labels, all_preds, average='binary')
    recall = recall_score(all_labels, all_preds, average='binary')
    f1 = f1_score(all_labels, all_preds, average='binary')

    metrics_df = pd.DataFrame({
        'Loss': [avg_loss],
        'Accuracy': [accuracy],
        'Precision': [precision],
        'Recall': [recall],
        'F1 Score': [f1]
    })

    cm = confusion_matrix(all_labels, all_preds)

    print(f"Test Loss: {avg_loss:.4f}, Test Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")
    return metrics_df, cm

## Training Loops

In [27]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_single_laplacians.to(device)
model_dual_lap_vr.to(device)
model_dual_lap_abstract.to(device)

DualInputCNN(
  (conv1_lap): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2_lap): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool_lap): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (adaptive_pool_lap): AdaptiveAvgPool2d(output_size=(100, 100))
  (conv1_pers): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2_pers): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (adaptive_pool_pers): AdaptiveAvgPool2d(output_size=(100, 100))
  (fc1): Linear(in_features=640000, out_features=128, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=128, out_features=2, bias=True)
)

In [34]:
criterion = nn.CrossEntropyLoss()
optimizer_single = optim.Adam(model_single_laplacians.parameters(), lr=0.001)
optimizer_dual_vr = optim.Adam(model_dual_lap_vr.parameters(), lr=0.001)
optimizer_dual_abstract = optim.Adam(model_dual_lap_abstract.parameters(), lr=0.001)

num_epochs = 10

In [35]:
# Initialize empty lists to store all metrics for each epoch
test_metrics_single_list = []
test_metrics_dual_vr_list = []
test_metrics_dual_abstract_list = []

# Initialize empty lists to store all confusion matrices for each epoch
cm_single_list = []
cm_dual_vr_list = []
cm_dual_abstract_list = []

# Training loop for single-input model (Laplacians)
for epoch in range(num_epochs):

      train_time_start = time.time()
      model_single_laplacians.train()  # Ensure the model is in training mode

      train_loss, train_acc = train_single_input(model_single_laplacians, train_loader_laplacians, optimizer_single, criterion, device)

      train_time_end = time.time()

      valid_time_start = time.time()

      model_single_laplacians.eval()  # Switch model to evaluation mode after training



      valid_loss, valid_acc = validate_single_input(model_single_laplacians, valid_loader_laplacians, criterion, device)

      valid_time_end = time.time()

      # Test the model after each epoch

      test_time_start = time.time()
      test_metrics_single, cm_single = test_single_input(model_single_laplacians, test_loader_laplacians, criterion, device)
      test_time_end = time.time()

      # Combine train, valid, and test metrics for this epoch
      metrics = test_metrics_single.copy()
      metrics['Train Loss'] = train_loss
      metrics['Train Accuracy'] = train_acc
      metrics['Valid Loss'] = valid_loss
      metrics['Valid Accuracy'] = valid_acc

      # Append to the list
      test_metrics_single_list.append(metrics)
      cm_single_list.append(cm_single)

      print(f"single-input model (Laplacians) Epoch {epoch+1}/{num_epochs}, "
            f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, "
            f"Valid Loss: {valid_loss:.4f}, Valid Acc: {valid_acc:.4f}")

      torch.save(model_single_laplacians.state_dict(), "Trained CNNs/model_single_h1_lap_nonunif_gudhi_epoch_{}.pth".format(epoch+1))

      print(f"single-input model (Laplacians)Training time: {train_time_end - train_time_start}")
      print(f"single-input model (Laplacians)Validation time: {valid_time_end - valid_time_start}")
      print(f"single-input model (Laplacians)Testing time: {test_time_end - test_time_start}")


# Training loop for dual-input model (Laplacians + VR Persistence Images)
for epoch in range(num_epochs):

      train_time_start = time.time()
      model_dual_lap_vr.train()  # Ensure the model is in training mode
      train_loss, train_acc = train_dual_input(model_dual_lap_vr, train_loader_laplacians, train_loader_vr, optimizer_dual_vr, criterion, device)
      train_time_end = time.time()
      valid_time_start = time.time()
      model_dual_lap_vr.eval()  # Switch model to evaluation mode after training


      valid_loss, valid_acc = validate_dual_input(model_dual_lap_vr, valid_loader_laplacians, valid_loader_vr, criterion, device)
      valid_time_end = time.time()

      # Test the model after each epoch
      test_time_start = time.time()
      test_metrics_dual_vr, cm_dual_vr = test_dual_input(model_dual_lap_vr, test_loader_laplacians, test_loader_vr, criterion, device)
      test_time_end = time.time()

      # Combine train, valid, and test metrics for this epoch
      metrics = test_metrics_dual_vr.copy()
      metrics['Train Loss'] = train_loss
      metrics['Train Accuracy'] = train_acc
      metrics['Valid Loss'] = valid_loss
      metrics['Valid Accuracy'] = valid_acc

      # Append to the list
      test_metrics_dual_vr_list.append(metrics)
      cm_dual_vr_list.append(cm_dual_vr)

      print(f"dual-input model (Laplacians + VR Persistence Images) Epoch {epoch+1}/{num_epochs}, "
            f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, "
            f"Valid Loss: {valid_loss:.4f}, Valid Acc: {valid_acc:.4f}")

      torch.save(model_dual_lap_vr.state_dict(), "Trained CNNs/model_dual_h1_lap_vr_nonunif_gudhi_epoch_{}.pth".format(epoch+1))

      print(f"dual-input model (Laplacians + VR Persistence Images) Training time: {train_time_end - train_time_start}")
      print(f"dual-input model (Laplacians + VR Persistence Images) Validation time: {valid_time_end - valid_time_start}")
      print(f"dual-input model (Laplacians + VR Persistence Images) Testing time: {test_time_end - test_time_start}")


# Training loop for dual-input model (Laplacians + Abstract Persistence Images)
for epoch in range(num_epochs):

      train_time_start = time.time()
      model_dual_lap_abstract.train()  # Ensure the model is in training mode
      train_loss, train_acc = train_dual_input(model_dual_lap_abstract, train_loader_laplacians, train_loader_abstract, optimizer_dual_abstract, criterion, device)
      train_time_end = time.time()

      valid_time_start = time.time()
      model_dual_lap_abstract.eval()  # Switch model to evaluation mode after training
      valid_loss, valid_acc = validate_dual_input(model_dual_lap_abstract, valid_loader_laplacians, valid_loader_abstract, criterion, device)
      valid_time_end = time.time()

      # Test the model after each epoch
      test_time_start = time.time()
      test_metrics_dual_abstract, cm_dual_abstract = test_dual_input(model_dual_lap_abstract, test_loader_laplacians, test_loader_abstract, criterion, device)
      test_time_end = time.time()

      # Combine train, valid, and test metrics for this epoch
      metrics = test_metrics_dual_abstract.copy()
      metrics['Train Loss'] = train_loss
      metrics['Train Accuracy'] = train_acc
      metrics['Valid Loss'] = valid_loss
      metrics['Valid Accuracy'] = valid_acc

      # Append to the list
      test_metrics_dual_abstract_list.append(metrics)
      cm_dual_abstract_list.append(cm_dual_abstract)

      print(f"dual-input model (Laplacians + Abstract Persistence Images) Epoch {epoch+1}/{num_epochs}, "
            f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, "
            f"Valid Loss: {valid_loss:.4f}, Valid Acc: {valid_acc:.4f}")

      torch.save(model_dual_lap_abstract.state_dict(), "Trained CNNs/model_dual_h1_lap_abstract_nonunif_gudhi_epoch_{}.pth".format(epoch+1))

      print(f"dual-input model (Laplacians + Abstract Persistence Images) Training time: {train_time_end - train_time_start}")
      print(f"dual-input model (Laplacians + Abstract Persistence Images) Validation time: {valid_time_end - valid_time_start}")
      print(f"dual-input model (Laplacians + Abstract Persistence Images) Testing time: {test_time_end - test_time_start}")


Training:   0%|          | 0/56 [00:00<?, ?it/s]

Validating:   0%|          | 0/7 [00:00<?, ?it/s]

Testing:   0%|          | 0/7 [00:00<?, ?it/s]

Test Loss: 0.0030, Test Accuracy: 0.9775, Precision: 0.9727, Recall: 1.0000, F1 Score: 0.9861
single-input model (Laplacians) Epoch 1/10, Train Loss: 0.0062, Train Acc: 0.9515, Valid Loss: 0.0056, Valid Acc: 0.9550
single-input model (Laplacians)Training time: 316.18478298187256
single-input model (Laplacians)Validation time: 9.177890062332153
single-input model (Laplacians)Testing time: 9.078251123428345


Training:   0%|          | 0/56 [00:00<?, ?it/s]

...

Testing (Dual Input):   0%|          | 0/7 [00:00<?, ?it/s]

Test Loss: 0.0016, Test Accuracy: 0.9955, Precision: 0.9944, Recall: 1.0000, F1 Score: 0.9972
dual-input model (Laplacians + Abstract Persistence Images) Epoch 9/10, Train Loss: 0.0047, Train Acc: 0.9741, Valid Loss: 0.1432, Valid Acc: 0.9685
dual-input model (Laplacians + Abstract Persistence Images) Training time: 346.5572190284729
dual-input model (Laplacians + Abstract Persistence Images) Validation time: 10.592430830001831
dual-input model (Laplacians + Abstract Persistence Images) Testing time: 10.20783805847168


Training (Dual Input):   0%|          | 0/56 [00:00<?, ?it/s]

Testing (Dual Input):   0%|          | 0/7 [00:00<?, ?it/s]

Test Loss: 0.0025, Test Accuracy: 0.9865, Precision: 0.9944, Recall: 0.9888, F1 Score: 0.9915
dual-input model (Laplacians + Abstract Persistence Images) Epoch 10/10, Train Loss: 0.0030, Train Acc: 0.9730, Valid Loss: 0.0574, Valid Acc: 0.9865
dual-input model (Laplacians + Abstract Persistence Images) Training time: 348.62528896331787
dual-input model (Laplacians + Abstract Persistence Images) Validation time: 9.825321912765503
dual-input model (Laplacians + Abstract Persistence Images) Testing time: 9.798104047775269


In [36]:
# Convert the lists of metrics into DataFrames with epoch as the index
test_metrics_single_df = pd.concat(test_metrics_single_list, ignore_index=True)
test_metrics_dual_vr_df = pd.concat(test_metrics_dual_vr_list, ignore_index=True)
test_metrics_dual_abstract_df = pd.concat(test_metrics_dual_abstract_list, ignore_index=True)

# Add 'Epoch' as new column
epochs = list(range(1, num_epochs + 1))

test_metrics_single_df['Epoch'] = epochs
test_metrics_dual_vr_df['Epoch'] = epochs
test_metrics_dual_abstract_df['Epoch'] = epochs

# Set 'Epoch' as index
test_metrics_single_df.set_index('Epoch', inplace=True)
test_metrics_dual_vr_df.set_index('Epoch', inplace=True)
test_metrics_dual_abstract_df.set_index('Epoch', inplace=True)

# Rename first two columns to 'Test Loss' and 'Test Accuracy'
test_metrics_single_df.rename(columns={'Loss': 'Test Loss', 'Accuracy': 'Test Accuracy'}, inplace=True)
test_metrics_dual_vr_df.rename(columns={'Loss': 'Test Loss', 'Accuracy': 'Test Accuracy'}, inplace=True)
test_metrics_dual_abstract_df.rename(columns={'Loss': 'Test Loss', 'Accuracy': 'Test Accuracy'}, inplace=True)

# Print final test metrics for all models
print("\nSingle-input model (Laplacians) Test Metrics:")
print(test_metrics_single_df)

print("\nDual-input model (Laplacians + VR Persistence Images) Test Metrics:")
print(test_metrics_dual_vr_df)

print("\nDual-input model (Laplacians + Abstract Persistence Images) Test Metrics:")
print(test_metrics_dual_abstract_df)



Single-input model (Laplacians) Test Metrics:
       Test Loss  Test Accuracy  Precision    Recall  F1 Score  Train Loss  \
Epoch                                                                        
1       0.003042       0.977477   0.972678  1.000000  0.986150    0.006248   
2       0.002991       0.977477   0.972678  1.000000  0.986150    0.005727   
3       0.003424       0.977477   0.972678  1.000000  0.986150    0.005490   
4       0.002778       0.977477   0.972678  1.000000  0.986150    0.005390   
5       0.002132       0.986486   0.983425  1.000000  0.991643    0.004645   
6       0.003079       0.981982   0.978022  1.000000  0.988889    0.004853   
7       0.001928       0.990991   0.988889  1.000000  0.994413    0.003833   
8       0.002229       0.981982   0.978022  1.000000  0.988889    0.003907   
9       0.002224       0.986486   0.983425  1.000000  0.991643    0.003979   
10      0.002073       0.986486   0.988827  0.994382  0.991597    0.003180   

       Train Acc

In [37]:
# Export test metrics to CSV files

test_metrics_single_df.to_csv('test_metrics_single_h1_lap_nonunif_gudhi.csv')
test_metrics_dual_vr_df.to_csv('test_metrics_dual_h1_lap_vr_nonunif_gudhi.csv')
test_metrics_dual_abstract_df.to_csv('test_metrics_dual_h1_lap_abstract_nonunif_gudhi.csv')

In [38]:
# Visualize and export confusion matrices to "Confusion\ Matrices/" folder

# Single-input model (Laplacians)
for i, cm in enumerate(cm_single_list):
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False, annot_kws={'size': 14})
    plt.title(f"Confusion Matrix - Single-input Model (Laplacians), Epoch {i+1}")
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.savefig(f"Confusion Matrices/single_h1_lap_nonunif_gudhi_cm_epoch_{i+1}.png")
    plt.close()

# Dual-input model (Laplacians + VR Persistence Images)
for i, cm in enumerate(cm_dual_vr_list):
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False, annot_kws={'size': 14})
    plt.title(f"Confusion Matrix - Dual-input Model (Laplacians + VR Persistence Images), Epoch {i+1}")
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.savefig(f"Confusion Matrices/dual_h1_lap_vr_nonunif_gudhi_cm_epoch_{i+1}.png")
    plt.close()

# Dual-input model (Laplacians + Abstract Persistence Images)
for i, cm in enumerate(cm_dual_abstract_list):
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False, annot_kws={'size': 14})
    plt.title(f"Confusion Matrix - Dual-input Model (Laplacians + Abstract Persistence Images), Epoch {i+1}")
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.savefig(f"Confusion Matrices/dual_h1_lap_abstract_nonunif_gudhi_cm_epoch_{i+1}.png")
    plt.close()